In [14]:
import sys
import os
import pandas as pd
import matplotlib
import numpy as np
from matplotlib.ticker import FuncFormatter
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, get_test_ipc2023_learning_instance_files, get_best_bound
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from util.visualise import *

In [15]:
# _PAPER_DIR=str(Path.home())+"/papers/manuscripts/24-icaps-kernels"
_ICAPS_PAPER_DIR=str(Path.home())+"/papers/manuscripts/icaps-24-kernels"

In [16]:
pd.options.display.float_format = "{:.1f}".format

In [39]:
_DIFFICULTIES = ["easy", "medium", "hard"]
_MODELS = [
  "blind",
  "hff",
  # "lama-seq",
#   "lama-first",
  "muninn", 
  "goose", 
  "ilg_1wl_1_0_linear-svr",
  # "linear-regression",
  "linear-svr", 
  "quadratic-svr", 
  "cubic-svr",
  "rbf-svr", 
  "ridge", 
  "lasso", 
  "mlp", 
]

WLS = {
    "ilg_1wl_1_0_linear-svr",
}
KERNELS = {"ridge", "lasso", "linear-svr", "quadratic-svr", "cubic-svr", "rbf-svr", "mlp"}
CLASSICAL = {"blind", "hff", "lama-first"}
GNNS = {"goose", "muninn"}

RENAME = {
    "ilg_1wl_1_0_linear-svr": "1wl"
}
def rename(model):
    if model in RENAME:
        return RENAME[model]
    return model

In [40]:
def get_data(domain):
    data = {}

    test_files = get_test_ipc2023_learning_instance_files()

    for model in _MODELS:
        d = {
            "solved": [],
            "expanded": [],
            "cost": [],
            "first_h": [],
            "quality": [],
            "c_opt": [],
            "difficulty": [],
            "time": [],
        }

        for difficulty in _DIFFICULTIES:
            for df, pf in test_files[domain]:
                if not difficulty in pf:
                    continue
                pf_name = os.path.basename(pf).replace(".pddl", "")
                stats = None

                if model == "muninn":
                    stats = {
                        "first_h": -1,
                        "solved": 0,
                        "time": -1,
                        "cost": -1,
                        "expanded": -1,
                        "evaluated": -1,
                    }

                    log_file = f"logs/logs_muninn/{domain}_{difficulty}_{pf_name}_domain-knowledge-{domain}.dk.log"

                    if not os.path.exists(log_file):
                        continue

                    """
                    [1] Subplan: 10, Heuristic: 9.998 -> goal, Expanded: 22, Generated: 86 (260 ms)
                    Writing file "plan.1" (10 actions)
                    Planned for 0 minutes and 0 seconds
                    """

                    for line in open(log_file, "r").readlines():
                        if "[1]" in line and stats["solved"] == 0:
                            toks = line.replace(",", "").replace("(", "").split()
                            stats["first_h"] = float(toks[4])
                        if "Writing file" in line:
                            toks = line.replace(",", "").replace("(", "").split()
                            stats["cost"] = int(toks[-2])
                            stats["solved"] = 1
                elif model == "goose":
                    dd = {
                        "easy": "0",
                        "medium": "1",
                        "hard": "2",
                    }[difficulty]
                    log_file = f"logs/gnn_ipc2023_test/{domain}:p-{dd}-{pf_name.replace('p', '')}:goose_fd.log"
                    assert os.path.exists(log_file), log_file
                elif model in KERNELS:
                    log_file = f"logs/opt_ipc2023_test/{domain}_{difficulty}_{pf_name}_-scratch-sv11-dc6693-trained_models_kernel-opt_{model}_ig_{domain}_wl_1_0.joblib.log"
                    # assert os.path.exists(log_file), log_file
                elif model in WLS:
                    log_file = f"icaps24_logs/{domain}_{difficulty}_{pf_name}_{domain}_{model}.log"
                else:
                    log_file = f"logs/ipc2023_{model}/{domain}_{difficulty}_{pf_name}_{model}.log"

                if stats is None:
                    if not os.path.exists(log_file):
                        continue
                    stats = scrape_search_log(log_file)

                for k in d:
                    if k in stats:
                        d[k].append(stats[k])

                c = stats["cost"] if stats["solved"] else -1
                c_opt = get_best_bound(domain, difficulty, pf_name)
                quality = c_opt / c
                quality = max(quality, 0)
                quality = min(quality, 1)
                d["quality"].append(quality)
                d["c_opt"].append(c_opt)
                d["difficulty"].append(difficulty)

        data[model] = d

    return data

In [41]:
def get_metrics_df(data):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  for config in _MODELS:
    d["config"].append(config)
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in _DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += 1
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [42]:
def get_table_df(metric="quality"):
    d = {
        "domain": [],
    }
    for config in _MODELS:
        d[rename(config)] = []

    sum_scores = {config: 0 for config in _MODELS}

    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        d["domain"].append(domain)

        for config in _MODELS:
            score = sum(data[config][metric])
            d[rename(config)].append(score)
            sum_scores[config] += score

    d["domain"].append("sum")
    for config in _MODELS:
        d[rename(config)].append(sum_scores[config])

    return pd.DataFrame(d)

In [43]:
scores_df = get_table_df(metric="solved")
scores = scores_df.to_numpy()
scores = scores[:,1:]
display(scores_df)

,domain,blind,hff,muninn,goose,1wl,linear-svr,quadratic-svr,cubic-svr,rbf-svr,ridge,lasso,mlp
0,blocksworld,8,28,39,29,15,15,23,17,17,18,7,11
1,childsnack,9,26,11,10,18,18,13,20,18,18,9,15
2,ferry,10,68,46,60,77,74,74,57,26,45,61,68
3,floortile,2,12,0,1,2,2,1,1,0,2,2,1
4,miconic,30,90,30,73,30,31,30,30,30,30,30,30
5,rovers,15,34,15,20,42,29,29,28,25,31,20,26
6,satellite,12,65,18,29,51,45,29,29,24,10,38,31
7,sokoban,27,36,27,30,38,38,33,31,30,38,26,37
8,spanner,30,30,32,30,30,30,30,30,30,30,30,30
9,transport,9,41,17,31,35,31,29,28,24,29,33,33


In [44]:
scores_df = get_table_df(metric="quality")
scores = scores_df.to_numpy()
scores = scores[:,1:]
display(scores_df)

,domain,blind,hff,muninn,goose,1wl,linear-svr,quadratic-svr,cubic-svr,rbf-svr,ridge,lasso,mlp
0,blocksworld,8.0,14.1,39.0,23.9,8.3,8.4,11.1,7.2,10.5,10.8,6.3,5.9
1,childsnack,9.0,20.1,11.0,8.7,17.2,17.2,12.5,18.7,17.2,13.9,9.0,14.4
2,ferry,10.0,67.6,46.0,59.7,76.6,73.6,73.6,56.8,25.7,38.3,60.6,67.6
3,floortile,2.0,11.2,0.0,0.9,1.8,1.8,0.9,0.9,0.0,1.8,1.8,0.9
4,miconic,30.0,88.5,30.0,72.0,28.6,29.8,28.5,28.7,28.5,28.2,29.9,28.9
5,rovers,15.0,32.7,14.2,17.7,31.3,20.1,22.4,21.2,20.5,23.4,19.0,18.7
6,satellite,12.0,63.8,18.0,24.1,44.1,35.7,25.3,25.3,19.8,4.6,36.3,23.9
7,sokoban,27.0,26.3,25.2,26.7,34.9,34.8,30.2,28.1,27.6,34.8,25.7,33.9
8,spanner,30.0,30.0,32.0,29.6,25.9,25.9,25.9,26.2,25.9,25.9,26.3,25.9
9,transport,9.0,39.3,17.0,29.0,28.6,26.1,26.3,25.2,22.2,23.9,32.2,27.4


In [23]:
""" coverage_tall.tex """
nnn = 0

scores_df = get_table_df()
scores = scores_df.to_numpy()
scores = scores[:,1:]

# ret  = r"\begin{tabular}{ c " + " ".join(["c" for _ in range(4*len(_MODELS))]) + r"} \toprule Domain "
ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(_MODELS))]) + r"} \toprule Domain "

for model in _MODELS:
    ret += f"& \\header{{\{model.replace('-', '')}}} "
ret += r"\\ \midrule" + '\n'

all_solved = {
  "easy": np.zeros(len(_MODELS)).astype(int),
  "medium": np.zeros(len(_MODELS)).astype(int),
  "hard": np.zeros(len(_MODELS)).astype(int),
  "all": np.zeros(len(_MODELS)).astype(int),
}
all_quality = {
  "easy": np.zeros(len(_MODELS)),
  "medium": np.zeros(len(_MODELS)),
  "hard": np.zeros(len(_MODELS)),
  "all": np.zeros(len(_MODELS)),
}

for domain in IPC2023_LEARNING_DOMAINS:
    ret += f"\\multirow{{4}}{{*}}{{{domain}}}"
    data = (get_metrics_df(get_data(domain)).to_numpy())

    # coverage then cost
    solved = {
        "easy": data[:,5].T,
        "medium": data[:,8].T,
        "hard": data[:,11].T,
        "all": data[:,2].T,
    }

    for difficulty in ["easy", "medium", "hard", "all"]:
        scores_domain = solved[difficulty]
        sorted_scores_domain = sorted(scores_domain, reverse=True)
        all_solved[difficulty] += np.array(scores_domain).astype(int)
        for i in range(len(solved["easy"])):
            chunks = []
            best_score = max(scores_domain)
            score = solved[difficulty][i]
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{score}}}")
            else:
                chunks.append(f"{score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    if domain==IPC2023_LEARNING_DOMAINS[-1]:
        ret += "\\\\ \midrule \n"
    else:
        ret += "\\\\ \cmidrule(lr){2-" + str(len(_MODELS)+1) + "} \n"

ret += f"\\multirow{{4}}{{*}}{{sum}}"

for difficulty in ["easy", "medium", "hard", "all"]:
    for i in range(len(all_solved["easy"])):
        chunks = []

        scores_domain = all_solved[difficulty]

        sorted_scores_domain = sorted(scores_domain, reverse=True)
        sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
        best_score = max(scores_domain)
        score = all_solved[difficulty][i]
        if score == 0:
            chunks.append("-")
        elif score==sorted_scores_domain[0]:
            chunks.append(f"\\first{{{score}}}")
        elif score==sorted_scores_domain[1]:
            chunks.append(f"\\second{{{score}}}")
        elif score==sorted_scores_domain[2]:
            chunks.append(f"\\third{{{score}}}")
        else:
            chunks.append(f"{score}")

        chunk = " & ".join(chunks)
        ret += f" & {chunk}"

    if difficulty != "all":
        ret += "\\\\ \n"

ret += r" \\ \bottomrule\end{tabular}"

for dir in [_ICAPS_PAPER_DIR]:
    f = open(f"{dir}/figures/coverage_tall.tex", 'w')
    f = open(f"{dir}/figures/coverage_tall.tex", 'w')
    f.write(ret)
    f.close()